In [ ]:
from bokeh.io import output_notebook
output_notebook()

# Analysis: Visualization

Finally we have some PC coefficients and scores, and a trained model.

__(Note: in this walkthrough we will be using 2 different sessions in order to create "crowd movies")__.

You will need ___more than one session recording___ in order to continue with this walkthrough.

Consider the following directory structure:

<img src="media/viz-dirstruct.png" alt="Starting directory structure" title="Starting Directory Structure" />

### Generate Index File

Using the __moseq2-viz__ tool, we can create an index file that will tell MoSeq2 where to find all the extractions and PCA results you wish to visualize.

Below are the options provided by the moseq2-viz tool.

In [ ]:
%%bash
moseq2-viz --help

<img src="media/viz-flags.png" title="Visualization Flag List" />

#### Note: in order to include all paths in the index file, run the following command one directory above the base.

In [ ]:
%%bash
moseq2-viz generate-index
mv moseq2-index.yaml sample_session

<img src="media/viz-yaml.png" alt="YAML File Example" title="Example of Generated YAML File" />

### Make Crowd Movies
This next command will enable you to generate "crowd movies" for your fit model; which are overlayed videos of your multiple recordings.

In [ ]:
%%bash
moseq2-viz make-crowd-movies moseq2-index.yaml my_model.p

<img src="media/viz-endstruct.png" alt="Ending directory structure" title="Final Directory Structure" />

## Crowd Video Example

Below is an example of a crowd video that you should see in your __crowd_movies/__ directory. It shows a number of mouse recordings overlayed into one clip, where a red dot appears on the mice for a split second indicating that all the mice in the recording have expressed the same syllable.
<img src="media/crowd_movie.gif" title="Crowd Movie" />

## Plotting Syllable Usages

The following command will generate plots that display the syllable usage distributions that the trained model can detect from your recordings.


In [ ]:
%%bash
cd sample_session/
moseq2-viz plot-usages moseq2-index.yaml my_model.p

### Syllable Usage Plot
Below is a plot that shows the probability that each syllable is being expressed; where the x-axis represents the syllables sorted by their label number, and the y-axis represents the probability value.
<img src="media/usages_1.png" title="Syllable Usages" />

## Plotting Multiple Groups

Plotting multiple groups may be useful when one would want to compare multiple different treatment groups. MoSeq2 offers an API to add and specify recordings to different groups such that you can discover different syllable usages and transition patterns in your specific control/treatment groups.

If you want to plot multiple groups, first you need to specify your groups+groupNames in your moseq2-index.yaml file. In the following example, we present a different YAML-index containing 6 mice recordings separated into 2 groups: group1 and group2. 

For your reference: paths with SubjectName: K are all in group1, while paths with SubjectName: I are in group2.

Following the YAML file are the commands used to create the YAML and plots shown respectively below:

Here, the metadata for each session, stored in metadata.json, comes in handy, since this is loaded into the metadata field for each file, which is now assigned a unique key (so if you move your data around we can keep track of it).

To achieve this specific YAML-index structure above, we ran the following bash commands:

(assuming you are in a directory containing multiple samples)

### Adding/Specifying Groups of Mice

You can use __moseq2-viz add-group__ to specify groups in the index.

Note: make sure you enter the correct corresponding SubjectName.

In [ ]:
%%bash
moseq2-viz generate-index
moseq2-viz add-group -k SubjectName -v K -g group1 moseq2-index.yaml
moseq2-viz add-group -k SubjectName -v K -g group2 -n moseq2-index.yaml

__For clarification, the second add-group command is specifying that SubjectNames != K are group2.__

For your convenience, below is a command line help manual depicting what some of the flags mean.

In [ ]:
%%bash
moseq2-viz add-group --help

<img src="media/add-group-help.png" title="Add Group Flags" />

MoSeq2-viz will search all of the sessions, and any sessions where the SubjectName is "K" will be assigned to group1. If you made a mistake and you want to change the metadata for a particular session, you can modify the yaml file directory.

### Plot Syllable Usages in Multiple Groups 

With the groups specified we can visualize things a few different ways. First, we can plot the syllable usages for each group (average and bootstrap confidence interval shown on a per session basis, i.e. one sample is one session):

Below, we are retraining our model such that it the recordings are associated with the correct group label in our above YAML.

__Note: the --separate-trans parameter is required in order to create separate transition matrices for your groups.__

In [ ]:
%%bash
cd _pca
moseq2-model learn-model -i ../moseq2-index.yaml -k 323859 --separate-trans pca_scores.h5 model_output.p

### Plot Scalars

Next we can plot a scalar summary which includes information such as velocity and average height:

You don't need to specify groups or a model here, by default each group is shown separately. 

In [ ]:
%%bash
cd ../
moseq2-viz plot-scalar-summary moseq2-index.yaml

<img src="media/viz-scalars.png" title="Final Directory Structure" />

### Plot Multi-Group Syllable Usages
For your reference, we have also included the syllable usages plot for multi-group comparison.
Once again, you can generate a usage plot with the following command _and the specificied flags_:

In [ ]:
moseq2-viz plot-usages moseq2-index.yaml _pca/model_output.p -g group1 -g group2

Creating the following usage plot:
<img src="media/usages.png" title="Syllable Usage Plot" />

### Scalars Summary Plot

Note that the following plots are derived from the latest (multi-group) index-YAML version shown above.

<img src="media/scalars_summary.png" title="Scalars Summary Plot" />


### Scalars Position (Tracking) Plot
<img src="media/scalars_position.png" title="Scalars Position Tracking Plot" />


### Plotting Transition Graphs

Finally, we can visualize the transition graphs and their differences with the following command:

__Note that we will use the model that has been trained on the separated groups in order to achieve the following plots.__

In [ ]:
%%bash
moseq2-viz plot-transition-graph moseq2-index.yaml _pca/model_output.p -g group1 -g group2

### Transition Graph for Difference Between Two Groups
<img src="media/transitions2.png" title="Transition Graph" />

If you only have one transition matrix group to graph, this is what the transition plot would look like:

### Transition Graph for One Group
<img src="media/transitions.png" title="Transition Graph" />

# Begin Syllable Statistical Testing

Now that we are done visualizing all of our data and results, we can start performing statistical analysis on the model's performance on unseen test data. [Click here for the Statistical Analysis walkthrough](http://localhost:8888/notebooks/MoSeq2_Step_6.ipynb) || [github link](https://github.com/dattalab/moseq2-docs/blob/ayman/usage-docs/MoSeq2_Step_6.ipynb)